In [1]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [2]:
# plotting
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [3]:
import pandas.io.data as web
#from pandas_datareader import data as web
import pandas_datareader as pdr

In [4]:
#pdr.get_data_yahoo('NIFTYBEES.NS')
NB = web.DataReader(name='NIFTYBEES.NS', data_source='yahoo',start='2008-1-1')
NB.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1925 entries, 2008-01-01 to 2016-07-15
Data columns (total 6 columns):
Open         1925 non-null float64
High         1925 non-null float64
Low          1925 non-null float64
Close        1925 non-null float64
Volume       1925 non-null int64
Adj Close    1925 non-null float64
dtypes: float64(5), int64(1)
memory usage: 105.3 KB


In [5]:
#Assumption: last day's US market close has impact on NIFTY
SPY = web.DataReader(name='SPY', data_source='yahoo',start='2008-1-1')
SPY.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2150 entries, 2008-01-02 to 2016-07-15
Data columns (total 6 columns):
Open         2150 non-null float64
High         2150 non-null float64
Low          2150 non-null float64
Close        2150 non-null float64
Volume       2150 non-null int64
Adj Close    2150 non-null float64
dtypes: float64(5), int64(1)
memory usage: 117.6 KB


In [6]:
NB['Change_'] = np.sign(np.log(NB['Open'] /NB['Close'].shift(1)))
NB['Change'] = np.log(NB['Open'] /NB['Close'].shift(1))
#NB['Return'] = np.log(NB['Close'] /NB['Close'].shift(1))

In [9]:
SPY['US_Mkt']= np.where(SPY['Close']>SPY['Close'].shift(1),1,np.where(SPY['Close']<SPY['Close'].shift(1),-1,0))
NB['US_Mkt']=SPY['US_Mkt'].shift(1)

In [8]:
NB['Volume']=NB['Volume']/pd.rolling_mean(NB['Volume'],10)

In [10]:
NB['Volatility']=NB['Close'].apply(lambda x: np.log(x))/pd.rolling_std(NB['Close'],21)

In [11]:
flagD=np.logical_and(np.logical_and(NB['Open'].shift(2)<NB['Close'].shift(3),NB['Open'].shift(1)< NB['Close'].shift(2)),NB['Open']< NB['Close'].shift(1))
flagU=np.logical_and(np.logical_and(NB['Open'].shift(2)>NB['Close'].shift(3),NB['Open'].shift(1)> NB['Close'].shift(2)),NB['Open']> NB['Close'].shift(1))

In [12]:
NB['3UD']= np.where(flagU,-1,np.where(flagD,1,0))

In [13]:
import TA# self made python file at PyCON16 folder for TA
#https://mrjbq7.github.io/ta-lib/func.html
import talib

In [14]:
#Calculating momentum of the close prices, with a time period of 5:
NB['momentum_'] = np.where(talib.MOM(np.asarray(NB['Close']), timeperiod=14)>50,1,np.where(talib.MOM(np.asarray(NB['Close']), timeperiod=14)<-50,-1,0))
NB['momentum'] = talib.MOM(np.asarray(NB['Close']), timeperiod=5)

In [15]:
NB['RSI_'] = np.where(talib.RSI(np.asarray(NB['Close']), timeperiod=14)>60,1,np.where(talib.RSI(np.asarray(NB['Close']), timeperiod=14)<45,-1,0))
NB['RSI'] = talib.RSI(np.asarray(NB['Close']), timeperiod=14)

In [16]:
  # reads data from Google Finance
NB['14d'] = pd.rolling_mean(NB['Close'], 14)
NB['42d'] = pd.rolling_mean(NB['Close'], 42)
  # 42d and 252d trends
NB['Cross']=np.where(NB['14d']>NB['42d'], 1,np.where(NB['14d']<NB['42d'],-1,0))

In [17]:
NB['BS2']=np.where(np.logical_and(np.logical_and(NB['momentum_']==1,NB['RSI_']==1),NB['Cross']==1),-1,np.where(np.logical_and(np.logical_and(NB['momentum_']==-1,NB['RSI_']==-1),NB['Cross']==-1),1,0))